-------------
# １．ライブラリのインポート

In [1]:
# 基本的なライブラリ
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from scipy import stats

# Scikit-learn関連
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.utils.validation import check_X_y
from sklearn.linear_model import Lasso

# 抽象基底クラス (ABC)
from abc import ABCMeta, abstractmethod

# タイピングのサポート
from typing import Optional

# 可視化の設定
import matplotlib.pyplot as plt
import japanize_matplotlib
import itertools

plt.style.use("ggplot")

import time

--------
# ２．実験設定

## 2.1 実験設定

In [2]:
### 実験データの設定 ###
n = 100000  # データサイズ
p = 20  # 特徴量数
s = 5  # 非ゼロ係数の数
rho = 0.35  # 相関レベル
snr = 0.5  # 信号対雑音比
beta_type = "type1"


### 実験設定 ###
SAMPLE_SIZE = 10000  # 標本サイズ
N_TRIALS = 10000  # 試行回数（標本抽出を行う回数）
m_VALUE = 2  # 最適標本配分における各クラスタの最小標本数
RANDOM_STATE = 0  # 乱数シード
EXP_NUM = 50

### 実験方法 ###
# クラスタリング
N_CLUSTERS = 6  # クラスタ数（Kmeans と GMM）


# 標本配分
ALLOCATION_LIST = [
    "Proportional",
    "Optimal",
]

### シード ###
np.random.seed(42)

In [3]:
if beta_type == "type1":
    BETA = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
    for i in range(p - 20):
        BETA.append(0)
    BETA = np.array(BETA)
    print(BETA)
ALL_FEATURES_INDEX = [i for i in range(p)]


[1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]


-----------
# ３．データの前処理

## 3.1 データ作成

In [4]:
def simulate_data(n, p, rho, snr, beta, seed):
    np.random.seed(seed)

    # 2. 共分散行列Σの生成
    Sigma = np.fromfunction(lambda i, j: rho ** np.abs(i - j), (p, p), dtype=int)

    # 3. Xの生成 (多変量正規分布)
    X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=n)

    # 4. 応答変数Yの生成
    signal_variance = beta.T @ Sigma @ beta  # β₀の分散
    noise_variance = signal_variance / snr  # ノイズ分散 σ²
    noise = np.random.normal(scale=np.sqrt(noise_variance), size=n)  # ノイズ項
    Y = X @ beta + noise

    return X, Y, beta

In [5]:
X_train, y_train, beta_train = simulate_data(
    n=n, p=p, rho=rho, snr=snr, beta=BETA, seed=0
)
X_test, y_test, beta_test = simulate_data(n=n, p=p, rho=rho, snr=snr, beta=BETA, seed=1)

## 3.5 標準化

In [6]:
sc = StandardScaler()
# 訓練データに基づいてfit
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

K-meansクラスタリング

In [7]:
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE)

TIME_LIST = []

# 各インスタンスに対して処理を実行
for exp_num in range(EXP_NUM):
    start_time = time.time()
    kmeans.fit(X_train_std)
    end_time = time.time()
    TIME_LIST.append(end_time - start_time)


In [8]:
print(f"{n}:{TIME_LIST}")

100000:[1.737678050994873, 0.28446125984191895, 0.30789685249328613, 0.2225513458251953, 0.269162654876709, 0.2215263843536377, 0.23646235466003418, 0.22952556610107422, 0.2241497039794922, 0.2251589298248291, 0.21365141868591309, 0.22915005683898926, 0.2011125087738037, 0.23912572860717773, 0.20899224281311035, 0.23643136024475098, 0.21433711051940918, 0.22634005546569824, 0.1999971866607666, 0.23313283920288086, 0.22189831733703613, 0.22333788871765137, 0.21254873275756836, 0.22532081604003906, 0.22540736198425293, 0.2160632610321045, 0.2263185977935791, 0.21089959144592285, 0.23500347137451172, 0.21699261665344238, 0.23022818565368652, 0.21134638786315918, 0.21440339088439941, 0.2124166488647461, 0.21637940406799316, 0.2118206024169922, 0.2169952392578125, 0.21625089645385742, 0.21925020217895508, 0.23118042945861816, 0.20463085174560547, 0.26600170135498047, 0.23145008087158203, 0.25171542167663574, 0.20077157020568848, 0.23181676864624023, 0.3356974124908447, 0.31162071228027344, 